In [ ]:
from model import Invicuna

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python import debug as tf_debug

import numpy as np
from viz import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

`num_bijectors = 0` corresponds to indentity map for the bijector. Set to 4 or 6 to learn a more interesting flow.

In [ ]:
config = {'num_bijectors': 0,
          'bijector': 'RealNVP',
          'network_size': 64,
          'use_batchnorm': False,
          'y_dim': 2,
          'batch_size': 20,
          'data_horizon': 10,
          'test_horizon': 10,
          'learning_rate': 1e-3}

In [ ]:
class gaussian_dataset:
    def __init__(self,dim,aux_dim):
        self.dim = dim
        self.aux_dim = aux_dim
    
    def sample(self,n_funcs=20,n_samples=20,get_mean=False):
        data = np.zeros((n_funcs,n_samples,self.dim + self.aux_dim))
        mu_list = []
        for i in range(n_funcs):
            mu = 3.0*( np.random.rand(self.dim) > 0.5 ) - 1.5
            mu_list.append(mu)
            
            for j in range(n_samples):
                data[i,j,:self.dim] = mu + 0.5*np.random.randn(self.dim)
                data[i,j,self.dim:] = np.random.randn(self.aux_dim)
        if get_mean:
            return mu_list,data
        
        return data

In [ ]:
g1 = tf.Graph()
sess1 = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=g1)

In [ ]:
model = Invicuna(sess1, config)
model.construct_model()

In [ ]:
dataset = gaussian_dataset(2,0)

In [ ]:
model.train(dataset,4000) 

In [ ]:
# generate conditioning data
mu, context_x = dataset.sample(n_funcs=1,n_samples=50,get_mean=True)


In [ ]:
prior_samples = dataset.sample(n_funcs=4, n_samples=500)
for j in range(4):
    plt.scatter(prior_samples[j,:,0], prior_samples[j,:,1], alpha=0.2)
plt.savefig('uncond_dataset.pdf')

In [ ]:
model.base.batch_shape

In [ ]:
analytic_model = analytic_gaussian_posterior()

In [ ]:
n_vals = [0,1,5,10]
N = len(n_vals)

#context_x = np.array([[[0.,1.5]]])

fig = plt.figure(figsize=[5*N,5])
for i,n in enumerate(n_vals):
    plt.subplot(1,N,i+1)
    plot_likelihood(mu,context_x[:,:n,:],model,x_range=[-3,3],y_range=[-3,3])
    
#     plt.subplot(N,2,2*i+2)
#     plot_likelihood(mu,context_x[:,:n,:],analytic_model,x_range=[-2,5],y_range=[-2,5])

plt.tight_layout()
plt.savefig('gaussian_fitting_noflow.pdf')
plt.show()

In [ ]:
model.get_likelihood(context_x,context_x)

In [ ]:
np.transpose(context_x,[1,0,2]).shape

In [ ]:
model.get_likelihood